In [59]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from StreamlitApp_S1.filtros_visualizaciones import p7_30_365_hist
import plotly.express as px
import folium

In [60]:
df_demanda = pd.read_csv('./SPRINT1PRUEBAS/Data/DF DEMANDA_20_25_LIMPIO_V1.csv')
df_generacion = pd.read_csv('./SPRINT1PRUEBAS/Data/DF GENERACION_20_25_LIMPIO_V1.csv')
df_intercambios = pd.read_csv('./SPRINT1PRUEBAS/Data/DF INTERCAMBIOS_20_25_LIMPIO_V1.csv')
df_balance = pd.read_csv('./SPRINT1PRUEBAS/Data/DF_BALANCE_20_25_LIMPIO_V1.csv')

### Demanda para 7, 30, 365 días y el histórico


In [6]:
today = datetime.now()
delta = timedelta(days=7)
start_date = (today - delta).strftime("%Y-%m-%dT%H:%M")
end_date = today.strftime("%Y-%m-%d")

df_demanda_filtered = df_demanda[(start_date <= df_demanda['fecha']) & (df_demanda['fecha']<= end_date)].reset_index(drop=True)
df_demanda_filtered.head()

,titulo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Demanda,0,2025-01-28,2025-02-19,Evolución de la demanda,nacional,716773.125,1.0,2025-02-14,2025,2,14,viernes,2025021921056
1,Demanda,0,2025-01-28,2025-02-19,Evolución de la demanda,nacional,651705.219,1.0,2025-02-15,2025,2,15,sábado,2025021921057
2,Demanda,0,2025-01-28,2025-02-19,Evolución de la demanda,nacional,599087.640,1.0,2025-02-16,2025,2,16,domingo,2025021921058
3,Demanda,0,2025-01-28,2025-02-19,Evolución de la demanda,nacional,703417.661,1.0,2025-02-17,2025,2,17,lunes,2025021921059
4,Demanda,0,2025-01-28,2025-02-19,Evolución de la demanda,nacional,729142.540,1.0,2025-02-18,2025,2,18,martes,2025021921060


### Intercambios

In [7]:
df_intercambios.head()


,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,frontera,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Exportación,export,0,2020-02-10,2025-02-19,Intercambios por frontera con Francia,Francia,nacional,-14842.623,0.531824,2020-01-01,2020,1,1,miércoles,202502190
1,Exportación,export,0,2020-02-10,2025-02-19,Intercambios por frontera con Francia,Francia,nacional,-16391.633,0.431717,2020-01-02,2020,1,2,jueves,202502191
2,Exportación,export,0,2020-02-10,2025-02-19,Intercambios por frontera con Francia,Francia,nacional,-1145.251,0.014396,2020-01-03,2020,1,3,viernes,202502192
3,Exportación,export,0,2020-02-10,2025-02-19,Intercambios por frontera con Francia,Francia,nacional,-2592.757,0.036120,2020-01-04,2020,1,4,sábado,202502193
4,Exportación,export,0,2020-02-10,2025-02-19,Intercambios por frontera con Francia,Francia,nacional,-1980.281,0.031696,2020-01-05,2020,1,5,domingo,202502194


In [8]:
df_intercambios['frontera'].unique()

array(['Francia', 'Portugal', 'Marruecos', 'Andorra',
       'Enlace Península-Baleares'], dtype=object)

In [9]:
today = datetime.strptime(df_intercambios['fecha'].max(), "%Y-%m-%d")
print(today)
delta = timedelta(days=7)
print(delta)
start_date = (today - delta).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")

print(start_date)
print(end_date)

2025-02-19 00:00:00
7 days, 0:00:00
2025-02-12
2025-02-19


In [61]:
df_intercambios_filtrado_1 = p7_30_365_hist(df_intercambios, 30)[0]
df_intercambios_filtrado_2 = df_intercambios_filtrado_1.groupby(['frontera','tipo'])['valor'].sum().reset_index()
df_intercambios_filtrado_2

,frontera,tipo,valor
0,Andorra,export,-11468.720
1,Andorra,import,0.000
2,Enlace Península-Baleares,export,-203545.482
3,Enlace Península-Baleares,import,-1.400
4,Francia,export,-480818.161
5,Francia,import,234806.459
6,Marruecos,export,-62437.392
7,Marruecos,import,11325.528
8,Portugal,export,-206799.985
9,Portugal,import,324271.830


In [62]:
exportacion_total = df_intercambios_filtrado_2.groupby('tipo').sum()['valor'].values[0]
importacion_total = df_intercambios_filtrado_2.groupby('tipo').sum()['valor'].values[1]

df_intercambios_filtrado_2.loc[len(df_intercambios_filtrado_2)] = ['Intercambios', 'export', exportacion_total]
df_intercambios_filtrado_2.loc[len(df_intercambios_filtrado_2)] = ['Intercambios', 'import', importacion_total]

In [63]:
df_intercambios_filtrado_2

,frontera,tipo,valor
0,Andorra,export,-11468.720
1,Andorra,import,0.000
2,Enlace Península-Baleares,export,-203545.482
3,Enlace Península-Baleares,import,-1.400
4,Francia,export,-480818.161
5,Francia,import,234806.459
6,Marruecos,export,-62437.392
7,Marruecos,import,11325.528
8,Portugal,export,-206799.985
9,Portugal,import,324271.830


In [ ]:
df_saldo = df_intercambios_filtrado_2.groupby('frontera').sum().reset_index()
df_saldo['tipo'] = df_saldo['tipo'].apply(lambda x: 'saldo')
df_saldo = pd.concat([df_intercambios_filtrado_2, df_saldo]).reset_index(drop=True)
df_saldo = df_saldo[df_saldo['frontera'].isin(coord_paises)]

,frontera,tipo,valor
0,Andorra,export,-11468.720
1,Andorra,import,0.000
2,Enlace Península-Baleares,export,-203545.482
3,Enlace Península-Baleares,import,-1.400
4,Francia,export,-480818.161
5,Francia,import,234806.459
6,Marruecos,export,-62437.392
7,Marruecos,import,11325.528
8,Portugal,export,-206799.985
9,Portugal,import,324271.830


In [74]:
coord_paises = {
                    "frontera": ["Intercambios", "Portugal", "Francia", "Andorra", "Marruecos"],
                    "latitud": [40.4168, 38.7169, 45.83362, 42.5078, 33.2514],
                    "longitud": [-3.7038, -9.1399, 1.24759, 1.5211, -5.372]
                }

df_coord = pd.DataFrame(coord_paises)
# df_intercambio_red = df_intercambios[['tipo', 'frontera', 'zona', 'valor', 'porcentaje', 'datetime']]
# df_intercambio_red = df_intercambio_red[df_intercambio_red['zona'] == 'nacional']
# df_intercambio_red['datetime'] = pd.to_datetime(df_intercambio_red['datetime'], utc=True)
# df_intercambio_total = df_intercambio_red.groupby(['frontera','type'])['value'].sum().reset_index()#.sort_values(by='value', ascending=False).reset_index(drop=True)
# df_intercambios_tot_coord = df_intercambio_total[df_intercambio_total['type'] == 'saldo'].reset_index(drop=True)
# df_intercambios_tot_coord = df_intercambios_tot_coord.merge(df_coord, on='frontera', how='left')
# df_intercambios_tot_coord

In [ ]:
df_saldo = df_intercambio_total.groupby('frontera').sum().reset_index()
df_saldo['tipo'] = df_saldo['tipo'].apply(lambda x: 'saldo')
df_saldo = pd.concat([df_intercambio_total, df_saldo])
df_saldo = df_saldo.merge(df_coord, on='frontera', how='left')
df_saldo

,frontera,tipo,valor,latitud,longitud
0,Andorra,saldo,-11468.720,42.50780,1.52110
1,Andorra,saldo,0.000,42.50780,1.52110
2,Enlace Península-Baleares,saldo,-203545.482,NaN,NaN
3,Enlace Península-Baleares,saldo,-1.400,NaN,NaN
4,Francia,saldo,-480818.161,45.83362,1.24759
5,Francia,saldo,234806.459,45.83362,1.24759
6,Marruecos,saldo,-62437.392,33.25140,-5.37200
7,Marruecos,saldo,11325.528,33.25140,-5.37200
8,Portugal,saldo,-206799.985,38.71690,-9.13990
9,Portugal,saldo,324271.830,38.71690,-9.13990


In [66]:
spain_map = folium.Map(location=[40.4637, -3.7492], tiles="openstreetmap", zoom_start=5, min_zoom=5, max_zoom=7, max_bounds=True)

geo_json_path = "./StreamlitApp_S1/world_countries.json"
# with open(geo_json_path, "r", encoding="utf-8") as file:
#     geo_json_data = json.load(file)

for pais, lat, lon in zip(df_coord['frontera'], df_coord['latitud'], df_coord['longitud']):

    print(pais)

    exp = df_saldo[(df_saldo['tipo'] == 'export') & (df_saldo['frontera'] == pais)]['valor']* 0.001
    imp = df_saldo[(df_saldo['tipo'] == 'import') & (df_saldo['frontera'] == pais)]['valor'] * 0.001
    saldo = df_saldo[(df_saldo['tipo'] == 'saldo') & (df_saldo['frontera'] == pais)]['valor'] * 0.001
    print(exp)

    popup_html = f"""
        <div style="width: 250px; font-size: 14px;">
        <b>📍{pais}</b><br>
        📤 Exportado: {round(exp, 1)} GWh<br>
        📥 Importado: {round(imp, 1)} GWh<br>
        ⚖️ Saldo: {round(saldo, 1)} GWh
        </div>
        """
    popup = folium.Popup(popup_html, max_width=300)
    if saldo.values[0] < 0:
        color = 'green'
    else:
        color = 'red'

    folium.Marker(
        location=[lat, lon],
        popup=popup,
        icon=folium.Icon(color=color, icon="bolt", prefix="fa")
        ).add_to(spain_map)

spain_map

Intercambios
Series([], Name: valor, dtype: float64)
Portugal
6   -206.799985
Name: valor, dtype: float64
Francia
2   -480.818161
Name: valor, dtype: float64
Andorra
0   -11.46872
Name: valor, dtype: float64
Marruecos
4   -62.437392
Name: valor, dtype: float64


In [34]:
spain_map = folium.Map(location = [40.4637, -3.7492], zoom_start = 5, min_zoom = 5)

coord = folium.map.FeatureGroup()

geo_json = "/Data/world_countries.json"

for pais, lat, lon in zip(df_coord["frontera"], df_coord["latitud"], df_coord["longitud"]):

    row = df_intercambio_total.loc[df_intercambio_total['frontera'] == pais].reset_index(drop=True)
    # print(row)
    exp = row[row['tipo'] == 'export']['valor'].values[0]
    # print(exp)
    imp = row[row['tipo'] == 'import']['valor'].values[0]
    # print(imp)
    saldo = row[row['tipo'] == 'saldo']['valor'].values[0]
    # print(tipo(saldo))

    popup_text = f"""
    <b>{pais}</b><br>
    📤 Exportado: {exp} MWh<br>
    📥 Importado: {imp} MWh<br>
    ⚖️ Saldo: {saldo} MWh<br>
    """
    if saldo < 0:
        color = 'green'
    else:
        color = 'red'
    

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color=color, icon="info-sign")
    ).add_to(spain_map)

folium.Choropleth(
        geo_data=geo_json,  
        data=df_intercambio_total,  
        columns=["frontera", "valor"], 
        key_on="feature.properties.ADMIN",  
        fill_color="BuGn",  
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Saldo Energético (MWh)",
).add_to(spain_map)

spain_map

IndexError: index 0 is out of bounds for axis 0 with size 0

### Generación


In [31]:
df_generacion_filtrado_24 = df_generacion[df_generacion['año'] == 2024]
df_generacion_filtrado_22 = df_generacion[df_generacion['año'] == 2022]

In [32]:
df_concat = pd.concat([df_generacion_filtrado_24, df_generacion_filtrado_22], ignore_index=True)
df_concat

,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,58381.148,0.097303,2024-01-01,2024,1,1,lunes,2025021987185
1,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,68004.167,0.092353,2024-01-02,2024,1,2,martes,2025021987186
2,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,93862.430,0.126579,2024-01-03,2024,1,3,miércoles,2025021987187
3,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,139605.013,0.211274,2024-01-04,2024,1,4,jueves,2025021987188
4,Hidráulica,Renovable,0,2025-01-28,2025-02-19,Generación por tecnología,nacional,83467.494,0.107495,2024-01-05,2024,1,5,viernes,2025021987189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38407,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,80701.361,0.057386,2022-08-01,2022,8,1,lunes,2025021965635
38408,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,52572.930,0.049885,2022-09-01,2022,9,1,jueves,2025021965636
38409,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,53683.588,0.054182,2022-10-01,2022,10,1,sábado,2025021965637
38410,Cogeneración,No renovable,0,2022-05-26,2025-02-19,Generación por tecnología,Región de Murcia,57330.527,0.082296,2022-11-01,2022,11,1,martes,2025021965638


In [70]:
df_intercambios_2y = df_intercambios[(df_intercambios['año'] == 2022) | (df_intercambios['año'] == 2024)]
df_intercambios_2y = df_intercambios_2y[df_intercambios_2y['frontera'].isin(['Francia', 'Portugal', 'Marruecos', 'Andorra'])].reset_index(drop=True)
df_intercambios_2y = df_intercambios_2y.groupby(['mes', 'tipo', 'año']).sum('valor').reset_index()
df_intercambios_2y['valor'] = df_intercambios_2y['valor'].apply(lambda x: abs(x))
df_intercambios_2y

,mes,tipo,año,composite,valor,porcentaje,dia,ID
0,1,export,2022,0,2055906.984,85.225440,1984,251102717255834
1,1,export,2024,0,1900027.114,81.347475,1984,251102718720522
2,1,import,2022,0,875929.211,38.774560,1984,251102717301094
3,1,import,2024,0,1410979.301,42.652525,1984,251102718769750
4,2,export,2022,0,1782735.936,78.159809,1624,226802454298896
5,2,export,2024,0,2173778.045,88.107905,1740,234902543322678
6,2,import,2022,0,1314824.592,33.840191,1624,226802454339776
7,2,import,2024,0,1053157.782,27.892095,1740,234902543368730
8,3,export,2022,0,1839121.201,85.857046,1984,251102717263150
9,3,export,2024,0,2159654.659,94.874299,1984,251102718727962


In [73]:
media = df_intercambios_2y['valor'].mean()
mediana = df_intercambios_2y['valor'].median()
maximo = df_intercambios_2y['valor'].max()
minimo = df_intercambios_2y['valor'].min()

fig = px.line(df_intercambios_2y, 
        x="mes", 
        y="valor", 
        color='tipo',
        line_dash="año",  # Diferenciar los años con estilos de línea
        markers=True, 
        title="Evolución Diaria de Importaciones y Exportaciones (2022 vs 2024)",
        labels={"valor": "Demanda (MWh)", "fecha": "Fecha"},
        color_discrete_map={"export": "green", "import": "red"}  # Colores personalizados
        )

fig.add_annotation(
    x=0.5, y=0.95, text=f"Media: {media:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
fig.add_annotation(
    x=0.5, y=0.90, text=f"Mediana: {mediana:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
fig.add_annotation(
    x=0.5, y=0.85, text=f"Máximo: {maximo:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
fig.add_annotation(
    x=0.5, y=0.80, text=f"Mínimo: {minimo:.2f}", showarrow=False, font=dict(size=12), align="center", bgcolor="rgba(255,255,255,0.7)"
)
        
                 

In [86]:
df_generacion

,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,151481.978,0.258134,2020-01-01,2020,1,1,miércoles,202502190
1,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,159171.675,0.229885,2020-01-02,2020,1,2,jueves,202502191
2,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,151996.921,0.226005,2020-01-03,2020,1,3,viernes,202502192
3,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,139873.002,0.209615,2020-01-04,2020,1,4,sábado,202502193
4,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Generación por tecnología,nacional,126801.179,0.210290,2020-01-05,2020,1,5,domingo,202502194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99018,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,118232.835,0.146870,2024-08-01,2024,8,1,jueves,20250219111167
99019,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,97220.279,0.148079,2024-09-01,2024,9,1,domingo,20250219111168
99020,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,72662.688,0.114139,2024-10-01,2024,10,1,martes,20250219111169
99021,Cogeneración,No renovable,0,2025-02-05,2025-02-19,Generación por tecnología,Región de Murcia,115075.762,0.171013,2024-11-01,2024,11,1,viernes,20250219111170


In [74]:
year1 = 2022
year2 = 2024

In [89]:
df_generacion_f = df_generacion[df_generacion['zona'] == 'nacional']
df_generacion_f = df_generacion_f[(df_generacion['año'] == year1) | (df_generacion['año'] == year2)]
# df_generacion_f['mes_dia'] = df_generacion_f['fecha'].apply(lambda x: x[5:11])
df_generacion_f = df_generacion_f.groupby(['mes', 'año']).sum('valor').reset_index()
df_generacion_f



C:\Users\armej\AppData\Local\Temp\ipykernel_20604\691024796.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,mes,año,composite,valor,porcentaje,dia,ID
0,1,2022,0,2.408670e+07,31.0,7440,941635205078418
1,1,2024,0,2.312715e+07,31.0,7440,941635225403072
2,2,2022,0,2.083565e+07,28.0,6090,850509217502574
3,2,2024,0,2.178124e+07,29.0,6525,880884565712698
4,3,2022,0,2.249587e+07,31.0,7440,941635205105853
5,3,2024,0,2.203221e+07,31.0,7458,943660247419405
6,4,2022,0,2.143581e+07,30.0,6975,911259875922615
7,4,2024,0,2.072055e+07,30.0,6975,911259895592085
8,5,2022,0,2.175847e+07,31.0,7440,941635205134218
9,5,2024,0,2.085135e+07,31.0,7440,941635225459337


In [90]:
fig = px.line(df_generacion_f, 
        x="mes", 
        y="valor", 
        color='año',
        markers=True, 
        title="Evolución de la generación (2022 vs 2024)",
        labels={"valor": "Demanda (MWh)", "fecha": "Fecha"},
        color_discrete_map={"export": "green", "import": "red"}  # Colores personalizados
        )

fig.show()

In [91]:
df_balance

,titulo,tipo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,151481.978,0.708451,2020-01-01,2020,1,1,miércoles,202502190
1,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,159171.675,0.589429,2020-01-02,2020,1,2,jueves,202502191
2,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,151996.921,0.572911,2020-01-03,2020,1,3,viernes,202502192
3,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,139873.002,0.463375,2020-01-04,2020,1,4,sábado,202502193
4,Hidráulica,Renovable,0,2021-01-12,2025-02-19,Balance de energía eléctrica,nacional,126801.179,0.544133,2020-01-05,2020,1,5,domingo,202502194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119549,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,82415.418,0.092869,2024-08-01,2024,8,1,jueves,20250219159036
119550,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,101140.254,0.133486,2024-09-01,2024,9,1,domingo,20250219159037
119551,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,101984.722,0.138078,2024-10-01,2024,10,1,martes,20250219159038
119552,Saldo de intercambios,Saldo de intercambios,0,2025-02-10,2025-02-19,Balance de energía eléctrica,Región de Murcia,49044.476,0.067933,2024-11-01,2024,11,1,viernes,20250219159039


In [94]:
df_demanda

,titulo,composite,ultima_actualizacion,fecha_extraccion,categoria,zona,valor,porcentaje,fecha,año,mes,dia,dia_semana,ID
0,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,574776.720000,1.0,2020-01-01,2020,1,1,miércoles,202502190
1,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,723522.375000,1.0,2020-01-02,2020,1,2,jueves,202502191
2,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,754645.342000,1.0,2020-01-03,2020,1,3,viernes,202502192
3,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,702390.878000,1.0,2020-01-04,2020,1,4,sábado,202502193
4,Demanda,0,2021-01-12,2025-02-19,Evolución de la demanda,nacional,651967.486000,1.0,2020-01-05,2020,1,5,domingo,202502194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18928,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,5.186531,1.0,2024-08-01,2024,8,1,jueves,2025021926435
18929,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,4.756270,1.0,2024-09-01,2024,9,1,domingo,2025021926436
18930,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,5.356519,1.0,2024-10-01,2024,10,1,martes,2025021926437
18931,Variación mensual corregida,1,2025-01-22,2025-02-19,Índice de Red Eléctrica General (IRE),nacional,-3.795579,1.0,2024-11-01,2024,11,1,viernes,2025021926438
